# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [89]:
import os
import glob
import psycopg2
import numpy as np
import pandas as pd
from sql_queries import *
import json

In [90]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()


In [ ]:
conn.close()

In [91]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [92]:
song_files = get_files('song_data')
song_files



['e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAAMO128F1481E7F.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAAMQ128F1460CD3.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAAPK128E0786D96.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAARJ128F9320760.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\song_data\\A\\A\\A\\TRAAAVG12903CFA5

In [93]:
df = pd.DataFrame()
for file in song_files:
    tmp_df = pd.read_json(file, lines=True)
    df = df.append(tmp_df)

df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
...,...,...,...,...,...,...,...,...,...,...
0,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008
0,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
0,1,ARVBRGZ1187FB4675A,NaN,NaN,,Gwen Stefani,SORRZGD12A6310DBC3,Harajuku Girls,290.55955,2004
0,1,ARWB3G61187FB49404,NaN,NaN,"Hamilton, Ohio",Steve Morse,SODAUVL12A8C13D184,Prognosis,363.85914,2000


In [155]:
# filepath = range(len(song_files))
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
...,...,...,...,...,...,...,...,...,...,...
0,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008
0,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
0,1,ARVBRGZ1187FB4675A,NaN,NaN,,Gwen Stefani,SORRZGD12A6310DBC3,Harajuku Girls,290.55955,2004
0,1,ARWB3G61187FB49404,NaN,NaN,"Hamilton, Ohio",Steve Morse,SODAUVL12A8C13D184,Prognosis,363.85914,2000


In [53]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
...,...,...,...,...,...,...,...,...,...,...
0,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008
0,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
0,1,ARVBRGZ1187FB4675A,NaN,NaN,,Gwen Stefani,SORRZGD12A6310DBC3,Harajuku Girls,290.55955,2004
0,1,ARWB3G61187FB49404,NaN,NaN,"Hamilton, Ohio",Steve Morse,SODAUVL12A8C13D184,Prognosis,363.85914,2000


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [94]:
filter_col = df[['song_id','title','artist_id','year','duration']].replace(" ", "")
song_data = filter_col.values.tolist()
song_data


[['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179],
 ['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546],
 ['SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, 177.47546],
 ['SONHOTT12A8C13493C',
  'Something Girls',
  'AR7G5I41187FB4CE6C',
  1982,
  233.40363],
 ['SOFSOCN12A8C143F5D',
  'Face the Ashes',
  'ARXR32B1187FB57099',
  2007,
  209.60608],
 ['SOYMRWW12A6D4FAB14',
  'The Moon And I (Ordinary Day Album Version)',
  'ARKFYS91187B98E58F',
  0,
  267.7024],
 ['SOMJBYD12A6D4F8557',
  'Keepin It Real (Skit)',
  'ARD0S291187B9B7BF5',
  0,
  114.78159],
 ['SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, 189.57016],
 ['SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, 269.81832],
 ['SOUDSGM12AC9618304',
  'Insatiable (Instrumental Version)',
  'ARNTLGG11E2835DDB9',
  0,
  266.39628],
 ['SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', 0, 218.77506],
 ['SOKEJEJ12A8C13E0D0',

In [85]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()


In [88]:
conn.close()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [96]:
for data in song_data:
    cur.execute(song_table_insert, data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [163]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
...,...,...,...,...,...,...,...,...,...,...
0,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008
0,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
0,1,ARVBRGZ1187FB4675A,NaN,NaN,,Gwen Stefani,SORRZGD12A6310DBC3,Harajuku Girls,290.55955,2004
0,1,ARWB3G61187FB49404,NaN,NaN,"Hamilton, Ohio",Steve Morse,SODAUVL12A8C13D184,Prognosis,363.85914,2000


In [98]:
art_col = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()
artist_data = art_col
artist_data

[['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan],
 ['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892],
 ['ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan],
 ['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan],
 ['ARXR32B1187FB57099', 'Gob', '', nan, nan],
 ['ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', nan, nan],
 ['ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan],
 ['AR10USD1187B99F3F1',
  'Tweeterfriendly Music',
  'Burlington, Ontario, Canada',
  nan,
  nan],
 ['AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan],
 ['ARNTLGG11E2835DDB9', 'Clp', '', nan, nan],
 ['ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962, -80.01955],
 ['ARC43071187B990240', 'Wayne Watson', 'Wisner, LA', nan, nan],
 ['ARL7K851187B99ACD2', 'Andy Andy', '', nan, nan],
 ['ARHHO3O1187B989413', 'Bob Azzam', '', nan, nan],
 ['ARIK43K1187B9AE54C', 'Lionel Richie', 'Beverly Hills, CA', nan, nan],
 ['ARD842G1187B997376',
  'Blue Rodeo',
  'T

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [99]:
for data in artist_data:
    cur.execute(artist_table_insert, data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [144]:
log_files = get_files('log_data')

In [39]:
# filepath = 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-01-events.json'

In [145]:
df = pd.DataFrame()
for file in log_files:
    tmp_df = pd.read_json(file, lines=True)
    df = df.append(tmp_df)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,1543603476796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
384,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,1543603678796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
385,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,1543603884796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
386,None,Logged In,Rylan,M,61,George,NaN,paid,"Birmingham-Hoover, AL",GET,Downgrade,1.541020e+12,1076,None,200,1543603993796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [146]:
# Convert ts to datetime. from milliseconds
# df.rename(columns = {'ts':'timestamp'}, inplace = True) # rename ts column to timestamp
df['ts'] = pd.to_datetime(df['ts'], unit='ms') # changes timstamp from ms to datetime timestamp
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,2018-11-01 20:57:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,2018-11-01 21:02:12.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,2018-11-30 18:44:36.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
384,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,2018-11-30 18:47:58.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
385,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,2018-11-30 18:51:24.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
386,None,Logged In,Rylan,M,61,George,NaN,paid,"Birmingham-Hoover, AL",GET,Downgrade,1.541020e+12,1076,None,200,2018-11-30 18:53:13.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


In [147]:
# Filter Dataframe based on NextSong being in page column
df = df[df.page.isin(['NextSong'])]
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Eriatarka,200,2018-11-01 21:11:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Becoming Insane,200,2018-11-01 21:17:33.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,Foo Fighters,Logged In,Rylan,M,57,George,271.38567,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Pretender,200,2018-11-30 18:40:05.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
383,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,2018-11-30 18:44:36.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
384,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,2018-11-30 18:47:58.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
385,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,2018-11-30 18:51:24.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


In [ ]:
df

In [148]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order

hour = df['ts'].dt.hour
day = df['ts'].dt.dayofweek
weekofyear = df['ts'].dt.weekofyear
month = df['ts'].dt.month
year = df['ts'].dt.year
time_data = [hour,day,weekofyear,month,year]
# time_data
column_labels = ('hour','day','weekofyear','month','year')


<ipython-input-148-987047c5a2d5>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  weekofyear = df['ts'].dt.weekofyear


In [149]:
column_data = {column_labels[0]: pd.Series(time_data[0]),
                column_labels[1]: pd.Series(time_data[1]),
                column_labels[2]: pd.Series(time_data[2]),
                column_labels[3]: pd.Series(time_data[3]),
                column_labels[4]: pd.Series(time_data[4])}
time_df = pd.DataFrame(column_data)
time_df

,hour,day,weekofyear,month,year
2,21,3,44,11,2018
4,21,3,44,11,2018
5,21,3,44,11,2018
6,21,3,44,11,2018
7,21,3,44,11,2018
...,...,...,...,...,...
382,18,4,48,11,2018
383,18,4,48,11,2018
384,18,4,48,11,2018
385,18,4,48,11,2018


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [150]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [151]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
...,...,...,...,...,...
382,16,Rylan,George,M,paid
383,16,Rylan,George,M,paid
384,16,Rylan,George,M,paid
385,16,Rylan,George,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [152]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [212]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
# conn.close()

In [ ]:
cur.execute(song_select)
cur.fetchone()

In [ ]:
df.head()

In [377]:
# conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
# cur = conn.cursor()
conn.close()


In [ ]:
# art_col = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
# ad = art_col.values.tolist()
# artist_data = ad
# artist_data
df['userAgent'] = df['userAgent'].str.strip('""')
df['userId'].replace(0,1)



In [153]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchall()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # # insert songplay record
    # data = df[['ts','userId', 'level', 'sessionId', ]]
    songplay_data = (row.ts, str(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    # conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.